# Exploratory Analysis of Spatial Data

3. create interactive visualization in bokeh
5. create interactive mask connection

### To-Do:

* fix data source input
* generate Choropleth of attribute
* make combined view
* include masking functionality in static 3 plot visualization

## Imports

In [72]:
%matplotlib inline 

import matplotlib.pyplot as plt
import pysal as ps
import libpysal.api as lp
import numpy as np
import pandas as pd

import geopandas as gpd
import os
import splot
import splot.plot
import bokeh
from pysal.contrib.pdio import read_files

In [ ]:
link_to_data = os.path.join('..', 'example_data', 'guerry', 'Guerry.shp')
df = gpd.read_file(link_to_data)

In [76]:
y = df['Donatns'].values
w = lp.Queen.from_dataframe(df)
w.transform = 'r'

In [77]:
import esda

w = lp.Queen.from_dataframe(df)
moran = esda.moran.Moran(y, w)
moran_loc = esda.moran.Moran_Local(y, w)
moran.I

0.3533613255848606

## Exploring Local Autocorrelation interactively with splot

In [78]:
from collections import OrderedDict
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool

from bokeh.io import show, output_notebook
from bokeh.models import GeoJSONDataSource, LinearColorMapper, CategoricalColorMapper

import geopandas as gpd
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [79]:
#obtain countries shapes
df = gpd.read_file(link_to_data)

geo_source = GeoJSONDataSource(geojson=df.to_json())

p = figure(title="France")

p.patches('xs', 'ys', fill_alpha=0.7, 
          fill_color={'field': 'Donatns', 'transform': LinearColorMapper(palette=palette)}, 
          line_color='black', line_width=0.5, source=geo_source)

show(p)

In [80]:
def plot_Lisa_Cluster_int(moran_loc, df, p=0.05): 
    
    # We're adding columns, so do that on a copy rather than on the users' input
    df = df.copy()
    
    # create a mask for local spatial autocorrelation
    sig = 1 * (moran_loc.p_sim < p)
    HH = 1 * (sig * moran_loc.q==1)
    LL = 3 * (sig * moran_loc.q==3)
    LH = 2 * (sig * moran_loc.q==2)
    HL = 4 * (sig * moran_loc.q==4)

    classes = HH + LL + LH + HL
    class_labels = [ 'ns', 'HH', 'LH', 'LL', 'HL']
    labels = [class_labels[i] for i in classes]
    # create a new column with label info
    df['labels'] = np.array(labels)

    colors5 = [ 'lightgrey', 'red', 'lightskyblue', 'mediumblue', 'pink']
    colors = [colors5[i] for i in classes]
   
    # Load the Tools
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    
    # load df into bokeh data source
    geo_source = GeoJSONDataSource(geojson=df.to_json())

    fig = figure(title="France", toolbar_location='right',
          plot_width=600, plot_height=500, tools=TOOLS)

    fig.patches('xs', 'ys', fill_alpha=0.8, 
              fill_color={'field': 'labels', 'transform': CategoricalColorMapper(palette=colors5, factors=class_labels)}, 
              line_color='white', line_width=0.5, source=geo_source)
    
    # add labels to figure (workaround, legend with geojsondatasource doesn't work,
    # see https://github.com/bokeh/bokeh/issues/5904)
    for factor, color in zip(class_labels, colors5):
        fig.patches(xs=[], ys=[], fill_color=color, legend=factor)

    return fig

fig = plot_Lisa_Cluster_int(moran_loc, df, p=0.05)
show(fig)

In [42]:
from bokeh.models import Span

y = df['Donatns'].values
w = lp.Queen.from_dataframe(df)
w.transform = 'r'
    
def mplot_interactive(moran_loc, p=0.05):    
    lag = ps.lag_spatial(moran_loc.w, moran_loc.z)
    fit = ps.spreg.OLS(moran_loc.z[:, None], lag[:,None])
    
    if p is not None:
        if not isinstance(moran_loc, esda.moran.Moran_Local):
            raise ValueError("`moran_loc` is not a Moran_Local instance")

        sig = 1 * (moran_loc.p_sim < p)
        HH = 1 * (sig * moran_loc.q==1)
        LL = 3 * (sig * moran_loc.q==3)
        LH = 2 * (sig * moran_loc.q==2)
        HL = 4 * (sig * moran_loc.q==4)
        classes = HH + LL + LH + HL
        
        class_labels = [ '0 ns', '1 HH', '2 LH', '3 LL', '4 HL']
        labels = [class_labels[i] for i in classes]

    colors5 = [ 'lightgrey', 'red', 'lightskyblue', 'mediumblue', 'pink']
    colors = [colors5[i] for i in classes]
    
    #Tools
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

    # Vertical line
    vline = Span(location=0, dimension='height', line_color='lightskyblue', line_width=2, line_dash = 'dashed')
    # Horizontal line
    hline = Span(location=0, dimension='width', line_color='lightskyblue', line_width=2, line_dash = 'dashed')
    
    fig = figure(title="Moran Scatterplot", x_axis_label='attribute', y_axis_label='Spatial Lag', toolbar_location='left', tools=TOOLS)
    fig.scatter(moran_loc.z, lag, color=colors, size=8, fill_alpha=.6)
    fig.renderers.extend([vline, hline])
    fig.xgrid.grid_line_color = None
    fig.ygrid.grid_line_color = None
    
    return fig
    

fig = mplot_interactive(moran_loc, p=0.05)
show(fig)


In [ ]:
'''
example coloration for Quantiles

bins_q5 = ps.Quantiles(HR90, k=5)

bwr = plt.cm.get_cmap('Reds')
bwr(.76)
c5 = [bwr(c) for c in [0.2, 0.4, 0.6, 0.7, 1.0]]
classes = bins_q5.yb
colors = [c5[i] for i in classes]'''

# Combined visualizations

Static visualization of Moran Scatterplot, LISA cluster map and choropleth map

## Back-up code options:

In [ ]:
# not a good way
def gpd_bokeh(df):
    """Convert geometries from geopandas to bokeh format"""
    nan = float('nan')
    lons = []
    lats = []
    for i, shape in enumerate(df.geometry.values):
        if shape.geom_type == 'MultiPolygon':
            gx = []
            gy = []
            ng = len(shape.geoms) - 1
            for j, member in enumerate(shape.geoms):
                xy = np.array(list(member.exterior.coords))
                xs = xy[:,0].tolist()
                ys = xy[:,1].tolist()
                gx.extend(xs)
                gy.extend(ys)
                if j < ng:
                    gx.append(nan)
                    gy.append(nan)
            lons.append(gx)
            lats.append(gy)

        else:     
            xy = np.array(list(shape.exterior.coords))
            xs = xy[:,0].tolist()
            ys = xy[:,1].tolist()
            lons.append(xs)
            lats.append(ys) 

    return lons,lats

In [ ]:
lons, lats = gpd_bokeh(df)

In [ ]:
def filter_nans(lons, lats):
    newlons = []
    newlats = []
    for lon, lat in zip(lons, lats):
        ix = np.isnan(lon) & np.isnan(lat)
        newlons.append(np.array(lon)[~ix])
        newlats.append(np.array(lat)[~ix])
        
    return newlons, newlats
        
newlons, newlats = filter_nans(lons, lats)
df['newlons'] = newlons
df['newlats'] = newlats

In [ ]:
Example for adding data to dataframe

In [ ]:
df['lons'] = np.array(lons)
df['lats'] = np.array(lats)

In [ ]:
lag = ps.lag_spatial(moran_loc.w, moran_loc.z)
df['lag'] = np.array(lag)